In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter

from transformer import Transformer, TOK_LEN, PAD_TOK, MAX_INST_LEN
from stacksyms import getFunctions, parseELF, parseDirectory

functions = []

#debugFilepath = 'data/cross-compile-dataset/bin/static/gcc/og/parallel'
#debugFiledir = 'data/cross-compile-dataset/bin/static/gcc/og'
debugFilepath = 'data/cross-compile-dataset/bin/static/gcc/og/sum'

In [2]:
#file2funcs = parseDirectory(debugFiledir)

functions = parseELF(debugFilepath)
#generateFeatures(functions)
#assert(not any(filter(lambda x : PAD_TOK in x, data)))

#main = list(filter(lambda x : x.name=='main', functions))[0]
#main_x = generateFeatures(main)


01/29/2021 23:21:28 INFO:parseELF:Trying to parse data/cross-compile-dataset/bin/static/gcc/og/sum as ELF
01/29/2021 23:21:28 INFO:parseDWARF:File has debug info..
01/29/2021 23:21:29 INFO:parseELF:Found 87 functions.


In [16]:
#print(sum(map(len,file2funcs.values())))
#print([max(map(len,fun.x)) for fun in functions]) # no of insts
#print(([sum(map(len,fun.x)) for fun in functions])) # no of toks
#X = [[tok + [PAD_TOK] * (16-len(tok)) for tok in func] for func in data] # quite the blow-up.. not good
#print(max(map(len,X[0])))
#print(functions[0].x[:10])
#print([[tok + [PAD_TOK] * (MAX_INST_LEN//TOK_LEN-len(tok)) for tok in func.x] for func in functions][:10])

12557


In [ ]:
# Training hyperparameters
num_epochs = 10000
learning_rate = 3e-4
batch_size = 32

# Model hyperparameters
embedding_size = 256
src_vocab_size = 2**16 # 2-byte sequence domain
trg_vocab_size = src_vocab_size
src_pad_idx = PAD_TOK # TODO: is this correct?
num_heads = 8
num_encoder_layers = 3
num_decoder_layers = 3
forward_expansion = 4
dropout = 0.10
max_len = MAX_INST_LEN

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', torch.cuda.get_device_name(0))

X = [[tok + [PAD_TOK] * (MAX_INST_LEN-len(inst)) for tok in func] for func in data]
x = torch.tensor(X) #.to(device)

loader = iter(DataLoader(ELFDataSet(x, y_train), batch_size=32, shuffle=True))

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data),
    batch_size=batch_size,
    sort_within_batch=True,
    sort_key=lambda x: len(x.src),
    device=device,
)

model = Transformer(
    embedding_size,
    src_vocab_size,
    trg_vocab_size,
    src_pad_idx,
    num_heads,
    num_encoder_layers,
    num_decoder_layers,
    forward_expansion,
    dropout,
    max_len,
    device,
).to(device)

optimizer = optim.Adam(model.parameters(), lr=learning_rate)

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, factor=0.1, patience=10, verbose=True
)

criterion = nn.CrossEntropyLoss(ignore_index=PAD_TOK)

# Tensorboard plots
writer = SummaryWriter("runs/loss_plot")
step   = 0

for epoch in range(num_epochs):
    print(f"[Epoch {epoch} / {num_epochs}]")
    model.eval()
    model.train()
    losses = []
    for batch_idx, batch in enumerate(train_iterator):
        inp_data = batch.src.to(device) # TODO fix
        target = batch.trg.to(device)   # TODO fix
        output = model(inp_data, target[:-1, :])
        # reshape and remove the start token
        output = output.reshape(-1, output.shape[2])
        target = target[1:].reshape(-1)
        optimizer.zero_grad()
        loss = criterion(output, target)
        losses.append(loss.item())
        loss.backward()
        # clip to avoid exploding gradient issues
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)
        optimizer.step() # stochastic gradient descent
        writer.add_scalar("Training loss", loss, global_step=step)
        step += 1
    mean_loss = sum(losses) / len(losses)
    scheduler.step(mean_loss)


In [19]:
from elftools.elf.elffile import ELFFile
from dwarf_import.model.module import Module
from dwarf_import.io.dwarf_import import DWARFDB, DWARFImporter, place_component_in_module_tree

elf = ELFFile(open(debugFilepath, 'rb'))
module = Module()
dwarfDB = DWARFDB(elf)
importer = DWARFImporter(dwarfDB, dict())
for component in importer.import_components():
    place_component_in_module_tree(module, component)

for cu in module.children():
    if type(cu) == Module:
        pass
    elif type(cu) == Component:
        pass
    print(type(cu))
#print(elf.get_section_by_name('.text').data())

<class 'dwarf_import.model.module.Module'>
<class 'dwarf_import.model.module.Module'>


In [ ]:
from stacksyms import parseDWARF
module, importer = parseDWARF(debugFilepath)
for cu in module.children():
    for component in cu.children():
        for func in component.functions:
            print(func)
    
dwarfDB = importer._dwarf_db
dwarfData = dwarfDB._pri
print(dir(dwarfData))
print(dwarfData._die_map)
# TODO: we could get basic block info from DWARF.. maybe
#       look at "objdump --dwarf=line execFile" and DWARF Spec Section 6.2
#       
#lineProg = dwarfData.get_line_program()
#dir(lineProg)
#types = firstUnit.types
#globals = firstUnit.variables
functions = firstUnit.functions
print([func.name for func in functions])
#scopeQueries = ['info scope ' + func for func in functions]
disasQueries = ['disas /r ' + func.name for func in functions]
gdbOut = staticGDB(debugFilepath, functions, disasQueries)
#scopeResults, disasResults = results[0:len(results)//2], results[len(results)//2:]

In [ ]:
# TODO: newCollectLocals(gdbOut, scopeQueries, functions) and newCollectDisas(gdbOut, disasQueries, functions)
#for scope, func in zip(scopeResults, functions):
#    collectLocals(scope, functions[func])
for disas, func in zip(gdbOut, functions):
    func.disas = [tuple(line.strip().split('\\t')) for line in disas[1:-1]]

In [ ]:
X, Y, Z = [], [], []
for func in functions:
    print('////////////////////////')
    print(func.name, hex(func.start), func.frame_base)
    for lvar in func.variables:
        print(lvar.name, lvar.type, "(bytesize = %d)"%lvar.type.byte_size)
        for loc in lvar.locations:
            print(hex(loc.begin) + " to " + hex(loc.end) + ": " \
            + str(loc.type)[13:]
            + str(loc.expr))
        print('')
#    X += [func] #generateFeature(func, functions)]
#    Y += [generateLabel(func, functions)]
#    Z += [generateDebugLabel(func, functions)]
#print([x+" => "+ str(y) for x,y in zip(X,Z)])

#func = 'quotearg_n_style_colon'

In [ ]:
for func in functions:
    print('////////////////////////')
    print(func.name, func.frame_base)
    for lvar in func.variables:
        print(lvar.name, lvar.type, "(bytesize = %d)"%lvar.type.byte_size)
        for loc in lvar.locations:
    #        print(loc)
            print(hex(loc.begin) + " to " + hex(loc.end) + ": " \
    #            + str(loc.type)[13:]
                + str(loc.expr))
            print('')

In [ ]:
generateLabel(func, functions)

In [ ]:
cu = ELFFile(open(debugFilepath, mode='rb')).get_dwarf_info().iter_CUs().__next__()